***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h1 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:50px;
               font-style:bold;
               color:white;">
        Part VI
    </h1>
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Crusader’s Journey in the World of Images Blog Series
    </h2>
</div>

***
by : JP Fabrero

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Importing Libraries
    </h2>
</div>

***

In [1]:
import os
os.environ['SKIMAGE_DATADIR'] = '/tmp/.skimage_cache'

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import skimage.io as skio
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from skimage.color import rgb2hsv

from pickling import *
from pyjanitor import auto_toc
toc = auto_toc()

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-fhmgfwxm because the default path (/home/jfabrero/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Preface
    </h2>
</div>

***
This a continuation of our previous endeavor but here, we're going to tackle Chromaticity Segmentation. Image differencing is also to be introduced as a method for efficiently comparing visual representations.

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Chromaticity Segmentation
    </h2>
</div>

***
Chromaticity-based segmentation focuses on the chromaticity or color information of pixels, while disregarding the intensity or brightness component. Chromaticity represents the color information relative to its intensity. By normalizing the color values based on intensity, chromaticity-based segmentation emphasizes the color properties of objects and reduces the influence of illumination changes.

In [2]:
image = imread('../mega_swampert.png')
fig, ax = plt.subplots(figsize=(15,8))
imshow(image)
ax.axis('off')
toc.add_fig('Sample Image (Mega Swampert)[1]', width=100)

In [3]:
def backproj(image, patch, bins=100, plotting=True):
    """Perform chromaticity based-segmentation"""
    # Convert to the HSV color space
    patch_hsv = rgb2hsv(patch[:,:,:3])
    img_hsv = rgb2hsv(image[:,:,:3])

    # Compute the 2D histogram of the reference patch in the H and S channels
    patch_hist, x, y = np.histogram2d(
        patch_hsv[:, :, 0].flatten(),
        patch_hsv[:, :, 1].flatten(),
        bins=100,
        range=[[0, 1], [0, 1]]
    )

    # Normalize the histogram
    patch_hist = patch_hist / np.max(patch_hist)

    # Compute the backprojection of the image histogram
    backproj = patch_hist[
        np.searchsorted(x, img_hsv[:, :, 0], side='left') - 1,
        np.searchsorted(y, img_hsv[:, :, 1], side='left') - 1
    ]

    # Normalize the backprojection
    backproj = backproj / np.max(backproj)

    # Grayscale image with 3 channels (the value is triplicated)
    img_gs = ((np.stack([rgb2gray(image[:,:,:3])] * 3, axis=-1) * 255)
              .astype('int').clip(0, 255))

    # Create a mask for skin
    mask = backproj > backproj.mean()
    img_seg = img_gs.copy()
    img_seg[mask] = image[:,:,:3][mask]

    fig, axs = plt.subplots(1, 3, figsize=(15, 6),
                            gridspec_kw=dict(width_ratios=(3,20,20)))
    if patch.shape[0] < 0.5*patch.shape[1]:
        patch = np.swapaxes(patch,0,1)
    axs[0].imshow(patch)
    axs[0].set_title('Patch Reference')
    
    axs[1].imshow(image)
    axs[1].axis('off')
    axs[1].set_title('Orignal Image')
    
    axs[2].imshow(img_seg)
    axs[2].axis('off')
    axs[2].set_title('Blue Segmented Image w/ Grayscale')
    for ax in axs:
        ax.axis('off')
    
    toc.add_fig('Sample of Chromaticity Segmentation', width=100)

In [4]:
backproj(image, image[400:500,450:550,:], 100)

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Image Differencing
    </h2>
</div>

***
Color image differencing is an image processing technique that involves comparing two or more color images to highlight the differences or changes in color between them. It is typically used to detect and analyze variations or movements in color over time or between different scenes. Color image differencing subtracts corresponding pixel values from different color images, resulting in a new image where the differences in color are emphasized. This technique is particularly useful in applications like motion detection, surveillance, and video analysis, where the detection of color changes provides valuable information for identifying objects or events of interest.

In [5]:
image2 = imread('../toy_story_1.png')
image3 = imread('../toy_story_2.png')
fig, ax = plt.subplots(1, 2, figsize=(15, 4))
ax[0].imshow(image2)
ax[0].axis('off')
ax[1].imshow(image3)
ax[1].axis('off')
toc.add_fig('Spot-the-Difference (Toy Story)[2]', width=100)

In [6]:
def img_diff(image_1, image_2):
    """Perform chromaticity based-segmentation"""
    image1_gray = rgb2gray(image_1[:,:,:3])
    image2_gray = rgb2gray(image_2[:,:,:3])
    
    diff = image1_gray - image2_gray
    
    plt.imshow(diff-diff.mean(), 'coolwarm', vmin=-0.15, vmax=0.15)
    plt.axis('off')
    toc.add_fig('Image Difference')
    
    return diff

In [7]:
diff = img_diff(image2, image3)

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Author's Notes
    </h2>
</div>

***
Non-parametric chromaticity segmentation is perhaps one of the most nifty tricks I learned in my journey. It has its own limitations but it is definitely a tool to take note of and use. Image differencing is a simple method but when iteratively, it builds up context and streamlines investigative process. It is important to note that these methods are more impactful when combined or built up in a pipeline.

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        References and Acknowledgement
    </h2>
</div>

***
The concepts discussed were derived from our MSDS2023 - IIP Course instructed by Benjur Emmanuel L. Borja. ChatGPT was used for concept reviews and writing prompts.

[1] Mega Swampert -Mega Evolve Art by Tomycase on DeviantArt. (2014, June 12). DeviantArt. https://www.deviantart.com/tomycase/art/Mega-Swampert-Mega-Evolve-Art-460315184

[2] Printable Disney Games. (n.d.). Disneyclips.com. https://www.disneyclips.com/funstuff/printables.html